# Analyse textuelle des publications Instagram des médias québécois en 2022-23

Cette analyse porte sur les publications Instagram plus récentes des médias du Québec (entre le 1<sup>er</sup> janvier 2022 et le 31 mai 2023).

Les différentes étapes ci-dessous correspondent à ce qui a été effectué sur le corpus de publications Facebook et qui sont décrites dans [cet autre carnet](FB_Analyse 2022-23.ipynb

In [49]:
import pandas as pan
import spacy, csv, numpy
from collections import Counter
tal = spacy.load("fr_core_news_sm")

In [12]:
insta = pan.read_csv("IG_22_23.csv", low_memory=False, index_col=[0])

In [13]:
insta

,Account,User Name,Followers at Posting,Post Created,Post Created Date,Post Created Time,Type,Total Interactions,Likes,Comments,...,Description,Image Text,Sponsor Id,Sponsor Name,Overperforming Score (weighted — Likes 1x Comments 1x ),Total Interactions (weighted — Likes 1x Comments 1x ),Overperforming Score,année,mois,inter
9887,TVA Abitibi-Témiscamingue,tvaabitibi,1182.0,2023-02-28 23:52:27 UTC,2023-02-28,23:52:27,Photo,0,0,0,...,"ÉCONOMIE | Malgré l'inflation, les contribuabl...",NaN,NaN,NaN,-12,NaN,NaN,2023,2023-02,0
9888,TVA Abitibi-Témiscamingue,tvaabitibi,1182.0,2023-02-28 23:51:46 UTC,2023-02-28,23:51:46,Photo,0,0,0,...,DISPARITIONS | Vous l'avez sans doute remarqué...,NOMBRE DE PERSONNES MINEURES DISPARUES DANS LA...,NaN,NaN,-12,NaN,NaN,2023,2023-02,0
9889,TVA Abitibi-Témiscamingue,tvaabitibi,1182.0,2023-02-28 23:51:02 UTC,2023-02-28,23:51:02,Photo,4,4,0,...,"RESTAURANT | Le phénomène des « no-show », soi...",NaN,NaN,NaN,"-0,33",NaN,NaN,2023,2023-02,4
9890,RDS,rds,122061.0,2023-02-28 23:50:30 UTC,2023-02-28,23:50:30,Photo,1 255,1246,9,...,"Malgré la surveillance de la LNH, Gustav Nyqui...",NaN,NaN,NaN,"-1,27",NaN,NaN,2023,2023-02,1255
9891,Rad,radpointca,102089.0,2023-02-28 23:25:05 UTC,2023-02-28,23:25:05,Album,2 316,2263,53,...,"Pour certains experts, la méfiance actuelle en...",NaN,NaN,NaN,"-1,22",NaN,NaN,2023,2023-02,2316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140537,Le Devoir,ledevoir,114619.0,2022-01-01 02:00:48 UTC,2022-01-01,02:00:48,Album,558,551,7,...,Alors que défilent les dernières heures de 202...,NaN,NaN,NaN,"1,31",NaN,NaN,2022,2022-01,558
140538,CityNews Montreal,citynewsmtl,15945.0,2022-01-01 01:15:38 UTC,2022-01-01,01:15:38,Video,21,19,2,...,From embarrassing Zoom slip-ups to dancing sen...,T.M City News FEEL-GOOD MOMENTS OF 2021 FRI 6:...,NaN,NaN,"1,05",NaN,NaN,2022,2022-01,21
140539,"104,7 Outaouais",1047outaouais,NaN,2022-01-01 01:08:49 UTC,2022-01-01,01:08:49,Photo,21,21,0,...,NaN,NaN,NaN,NaN,"-1,81",NaN,NaN,2022,2022-01,21
140540,Radio-Canada Sports,rcsports,16140.0,2022-01-01 00:30:12 UTC,2022-01-01,00:30:12,Video,103,103,0,...,On se sauve de 2021 et on saute pleinement ver...,NaN,NaN,NaN,"1,18",NaN,NaN,2022,2022-01,103


In [15]:
insta.groupby(by="mois").Account.nunique()

mois
2022-01    73
2022-02    74
2022-03    75
2022-04    75
2022-05    74
2022-06    76
2022-07    70
2022-08    73
2022-09    73
2022-10    71
2022-11    69
2022-12    70
2023-01    69
2023-02    67
2023-03    67
2023-04    67
2023-05    68
Name: Account, dtype: int64

In [50]:
numpy.sort(insta.mois.unique())

array(['2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
       '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
       '2023-01', '2023-02', '2023-03', '2023-04', '2023-05'],
      dtype=object)

In [18]:
f = open("IG_22_23.csv")
pubs = csv.reader(f)
next(pubs)

['',
 'Account',
 'User Name',
 'Followers at Posting',
 'Post Created',
 'Post Created Date',
 'Post Created Time',
 'Type',
 'Total Interactions',
 'Likes',
 'Comments',
 'Views',
 'Like and View Counts Disabled',
 'URL',
 'Link',
 'Photo',
 'Title',
 'Description',
 'Image Text',
 'Sponsor Id',
 'Sponsor Name',
 'Overperforming Score (weighted  —  Likes 1x Comments 1x )',
 'Total Interactions (weighted  —  Likes 1x Comments 1x )',
 'Overperforming Score',
 'année',
 'mois',
 'inter']

In [57]:
tal.max_length = 1500000

In [60]:
motsNot = ["the", "in", "to", "and", "of", "|", "for", "s", "t", "full", "story", "tap", "click", "by", "be",
           "will", "link", "more", "at", "our", "follow", "us", "don'", "forget", "website", "like", "-t",
           "instagramdiv", "is", "do", "it", "he", "she", "we"]

In [61]:
for mois in numpy.sort(insta.mois.unique()):
    
    f = open("IG_22_23.csv")
    pubs = csv.reader(f)
    next(pubs)
    
    texte = ""
    for pub in pubs:
        if pub[-2] == mois:
            texte = texte + pub[17] + " " + pub[18] + " "
#             print(pub[17])
#             print(pub[18])
#             print("***")
    
#     print(mois, len(texte))
            
    texte = texte.replace("\n", " ").replace("’", "'").replace("\xa0", " ").replace("\u2060", " ").replace("\u2009", " ").replace("\u2063", " ")
    
    while "  " in texte:
        texte = texte.replace("  "," ").strip()
    print(mois,len(texte))
    
    doc = tal(texte)
# #     print(len(doc))
    
    lemmes = [token.lemma_.lower() for token in doc if token.is_stop == False and token.is_punct == False and token.lemma_.lower() not in motsNot and len(token.text) > 1 and "/" not in token.text and "@" not in token.text and token.text != "-t"]
#     print(mois,len(lemmes))
    
    freq = Counter(lemmes)
    print(freq.most_common(10))
    
    for n in freq.most_common():
#         print([jour[:7],n[0],n[1]])
        daft = open("IG_motsSeuls.csv", "a")
        punk = csv.writer(daft)
        punk.writerow([mois,n[0],n[1]])
    
    bigrammes = []
    
    for index, lemme in enumerate(lemmes[:-1]):
        bigrammes.append("{} {}".format(lemmes[index], lemmes[index+1]))
    
    freq = Counter(bigrammes)
    print(freq.most_common(10))
    
    for n in freq.most_common():
#         print([jour[:7],n[0],n[1]])
        daft = open("IG_bigrammes.csv", "a")
        punk = csv.writer(daft)
        punk.writerow([mois,n[0],n[1]])
    
    emojis = []

    for token in doc:
        if token.is_stop == False and token.is_punct == False and len(token.text) == 1:
            emojis.append(token.text)
    
    freq = Counter(emojis)
    print(freq.most_common(10))
    
    for n in freq.most_common():
#         print([jour[:7],n[0],n[1]])
        daft = open("IG_emojis.csv", "a")
        punk = csv.writer(daft)
        punk.writerow([mois,n[0],n[1]])
    
    hashtags = []

    for token in texte.split():
        if token[0] == "#":
            hashtags.append(token)
    
    freq = Counter(hashtags)
    print(freq.most_common(10))
    
    for n in freq.most_common():
#         print([jour[:7],n[0],n[1]])
        daft = open("IG_hashtags.csv", "a")
        punk = csv.writer(daft)
        punk.writerow([mois,n[0],n[1]])

2022-01 943549
[('bio', 1228), ('lien', 954), ('savoir', 605), ('québec', 475), ('quebec', 421), ('clique', 412), ('an', 332), ('être', 302), ('photo', 298), ('montreal', 273)]
[('lien bio', 854), ('bio savoir', 500), ('clique lien', 412), ('cliquer lien', 170), ('cjad800.com bio', 91), ('bio photo', 89), ('savoir instagram', 86), ('city news', 76), ('lien disponible', 69), ('disponible bio', 69)]
[('📸', 689), ('|', 571), ('>', 378), ('s', 273), ('📷', 241), ('t', 165), ('️', 131), ('<', 111), ('$', 99), ('°', 90)]
[('#quebec', 173), ('#ledevoir', 110), ('#montreal', 106), ('#covid19', 89), ('#hiver', 80), ('#rcinfo', 80), ('#covid19qc', 69), ('#coronavirus', 60), ('#polqc', 60), ('#environnement', 57)]
2022-02 1077617
[('bio', 1329), ('lien', 1014), ('savoir', 573), ('québec', 459), ('clique', 406), ('quebec', 395), ('photo', 345), ('an', 329), ('olympique', 318), ('montreal', 304)]
[('lien bio', 928), ('bio savoir', 456), ('clique lien', 406), ('cliquer lien', 221), ('jeu olympique', 

In [73]:
table = pan.pivot_table(insta, values=["inter", "URL", "Followers at Posting", "Views", "Comments", "Likes"], aggfunc={"URL": "count", "inter": numpy.sum, "Followers at Posting": numpy.max, "Views": numpy.sum, "Comments": numpy.sum, "Likes": numpy.sum}, index="Account")
# table = pan.pivot_table(insta, values=["inter", "URL"], aggfunc="count", index="Account")
table

,Comments,Followers at Posting,Likes,URL,Views,inter
Account,,,,,,
"104,7 Outaouais",31,2656.0,491,15,0,522
24 heures,5114,14956.0,209744,678,404214,214858
98.5 Montréal,115,10545.0,1732,14,13737,1847
Beauce Media,8,1278.0,197,26,1823,205
Bible urbaine,393,2688.0,6491,302,210,6884
...,...,...,...,...,...,...
TVA Sports,17482,54120.0,928290,1215,918963,945772
URBANIA,9602,80774.0,349308,472,5677126,358910
Unpointcinq,118,2880.0,7383,280,1128,7501


In [71]:
insta.groupby(by="Account").Type.value_counts().to_csv("IG_types.csv")

In [72]:
insta.groupby(by="Account").Type.value_counts()

Account          Type 
104,7 Outaouais  Photo     14
                 Album      1
24 heures        Album    544
                 Photo     69
                 Video     65
                         ... 
lavoixdelest     Video     14
                 Photo      9
porte-monnaie    Album     99
                 Video      6
                 Photo      5
Name: Type, Length: 222, dtype: int64

In [74]:
table.to_csv("IG_parMédia.csv")